# CPHASE gate between a transmon and a fluxonium using higher levels of the fluxonium: leakage analysis
In this notebook we study leakage in the two-qubit gate between a transmon and a fluxonium inspired by the two-qubit gate between a transmon and a fluxonium described in Ficheux et al. Phys. Rev. X 11, 021026 (2021). See notebook "gate_setup_and_fidelity.ipynb" for more details about the gate principle.

In [1]:
import numpy as np
import time
import qutip as qtp
import matplotlib.pyplot as plt
import scipy.integrate
from scipy.optimize import minimize
import pysqkit
from pysqkit import QubitSystem
from pysqkit.util.metrics import average_process_fidelity, \
    average_gate_fidelity
from pysqkit.drives.pulse_shapes import gaussian_top
from pysqkit.util.phys import temperature_to_thermalenergy
from pysqkit.util.quantum import generalized_rabi_frequency
import pysqkit.util.transformations as trf
from pysqkit.util.hsbasis import weyl_by_index
from pysqkit.solvers.solvkit import integrate
from typing import List, Dict, Callable
import matplotlib
matplotlib.rcParams['mathtext.fontset'] = 'cm'
import util_cphase as util
import cmath
import json
import multiprocessing
from functools import partial

from IPython.display import display, Latex


From a first analysis it seems that leakage is mainly due to incoherent processes in the fluxonium. In fact, adding relaxation to the fluxonium highly increases the leakage. 

In [2]:
with open('../flx_transm_params.txt') as param_file:
    parameters_set = json.load(param_file)

In [17]:
temperature = 0.020 # K
thermal_energy = temperature_to_thermalenergy(temperature) # kb T/h in GHz
d_comp = 4

p_set = "0"


#Transmon
levels_t = 3
transm = pysqkit.qubits.SimpleTransmon(
    label='T', 
    max_freq=parameters_set[p_set]["max_freq_t"], 
    anharm=parameters_set[p_set]["anharm_t"],
    diel_loss_tan=parameters_set[p_set]["diel_loss_tan_t"], #set to zero to check d_1 L1 = d_2 L2
    env_thermal_energy=thermal_energy,    
    dim_hilbert=levels_t,
    dephasing_times=None #parameters_set[p_set]["dephasing_times_t"]
)

#Fluxonium
levels_f = 5

flx = pysqkit.qubits.Fluxonium(
    label='F', 
    charge_energy=parameters_set[p_set]["charge_energy_f"], 
    induct_energy=parameters_set[p_set]["induct_energy_f"], 
    joseph_energy=parameters_set[p_set]["joseph_energy_f"], #8.0, 
    diel_loss_tan=parameters_set[p_set]["diel_loss_tan_f"], #set to zero to check d_1 L1 = d_2 L2
    env_thermal_energy=thermal_energy,
    dephasing_times=None #parameters_set[p_set]["dephasing_times_f"] #ns/2*np.pi 
)
flx.diagonalize_basis(levels_f)

# We also add a drive on the fluxonium
flx.add_drive(
    pysqkit.drives.microwave_drive,
    label='cz_drive_f',
    pulse=pysqkit.drives.pulses.cos_modulation,
    pulse_shape=pysqkit.drives.pulse_shapes.gaussian_top
)

d_leak = levels_t*levels_f - d_comp

jc = parameters_set[p_set]["jc"]
coupled_sys = transm.couple_to(flx, coupling=pysqkit.couplers.capacitive_coupling, strength=jc)
bare_system = transm.couple_to(flx, coupling=pysqkit.couplers.capacitive_coupling, strength=0.0)

In [18]:
state_label = ["00", "01", "10", "11"]
comp_states = {}
for label in state_label:
    state_tmp = coupled_sys.state(label)[1]
    loc = np.argmax(np.abs(state_tmp))
    phase = cmath.phase(state_tmp[loc])
    state_tmp = np.exp(-1j*phase)*state_tmp
    comp_states[label] = state_tmp
comp_states_list = []
for key in comp_states.keys():
    comp_states_list.append(comp_states[key])

In [19]:
def func_to_minimize(
    x0: np.ndarray,
    levels_first_transition: List['str'],
    levels_second_transition: List['str'],
    system: pysqkit.systems.system.QubitSystem,
    cond_phase: float,
    eps_ratio_dict: Dict    
) -> float:
    
    """
    Description
    --------------------------------------------------------------------------
    Function to minimize in order to match the parameters to 
    implement a CPHASE gate given a certain conditional phase up to 
    single-qubit rotations. 

    Parameters
    --------------------------------------------------------------------------
    x0 : np.ndarray([eps_reference, drive_freq]) 
        It represents the parameters to be minimized.
    levels_first_transition : List['str'] 
        List with the labels of the first transition whose generalized Rabi 
        frequency has to be matched
    levels_second_transition : List['str'] 
        List with the labels of the second transition whose generalized Rabi 
        frequency has to be matched
    system: QubitSystem
        The coupled system we are analyzing
    cond_phase: float
        Conditional phase
    eps_ratio_dict: Dict 
        Dictionary whose keys are system.labels. The entries correspond
        to the ratios between the corresponding qubit drive and the 
        reference drive.     
    """
    
    qubit_labels = system.labels
    eps = {}
    for qubit in qubit_labels:
        eps[qubit] = x0[0]*eps_ratio_dict[qubit]
    rabi_first_transition = generalized_rabi_frequency(levels_first_transition, eps, x0[1], system)
    rabi_second_transition = generalized_rabi_frequency(levels_second_transition, eps, x0[1], system)
    delta_gate = util.delta(system)
    y = np.sqrt( (rabi_first_transition - rabi_second_transition)**2 + \
                rabi_first_transition**2*(cond_phase - delta_gate/rabi_first_transition*np.pi)**2)
    return np.abs(y/delta_gate)

def func_to_minimize_time(
    pulse_time: list,
    t_rise: float,
    rabi_period
) -> float:
    step = 1e-3
    n_points = int(pulse_time[0]/step)
    times = np.linspace(0, pulse_time[0], n_points)
    pulse = gaussian_top(times, t_rise, pulse_time[0])
    integral = scipy.integrate.simpson(pulse, times)
    return np.abs(integral - rabi_period) 

In [20]:
cond_phase = np.pi
qubit_labels = coupled_sys.labels
x0 = np.array([0.017, 7.15]) #initial guess 
eps_ratios = {qubit_labels[0]: 0.0, qubit_labels[1]:1.0}
args_to_pass = (['00', '03'], ['10', '13'], coupled_sys, cond_phase, eps_ratios) 
minimization_result = minimize(func_to_minimize, x0, args=args_to_pass)
eps_drive = minimization_result['x'][0]
eps_drive = minimization_result['x'][0]
freq_drive = minimization_result['x'][1]

eps = {}
for qubit in qubit_labels:
    eps[qubit] = eps_drive*eps_ratios[qubit]
rabi_period = 1/generalized_rabi_frequency(["00", "03"], 
                                           eps, freq_drive, coupled_sys)

t_rise = 10.0 # [ns]


t_gate_0 = [rabi_period]

args_to_pass = (t_rise, rabi_period) 

minimization_result_time = minimize(func_to_minimize_time, 
                                    t_gate_0, args=args_to_pass)

pts_per_drive_period = 10
t_gate_ideal = minimization_result_time['x'][0]
nb_points = int(t_gate_ideal*freq_drive*pts_per_drive_period)
t_list = np.linspace(0, t_gate_ideal, nb_points)

In [21]:
coupled_sys['F'].drives['cz_drive_f'].set_params(phase=0, time=t_list, 
                                                 rise_time=t_rise, pulse_time=t_gate_ideal,
                                                 amp=eps_drive, freq=freq_drive)

In what follows we are interested in studying leakage out of the computational subspace that we here define

In [22]:
state_labels = ['00', '01', '10', '11']
comp_states = []
for label in state_labels:
    comp_states.append(coupled_sys.state(label)[1])

In [23]:
def run(
    time: np.ndarray,
    system: QubitSystem, 
    state_init: qtp.qobj.Qobj,
    with_noise: bool=True,
    options: qtp.solver.Options=None
):
    hamil0 = system.hamiltonian(as_qobj=True)
    hamil_drive = []
    pulse_drive = []
                    
    for qubit in system:
        if qubit.is_driven:
            for label, drive in qubit.drives.items():
                hamil_drive.append(drive.hamiltonian(as_qobj=True))
                pulse_drive.append(drive.eval_pulse())
    if with_noise:
        jump_op = [op for qubit in system for op in qubit.collapse_ops(as_qobj=True)]
    else:
        jump_op = []
                    
    result = integrate(time, state_init, hamil0, hamil_drive,
                           pulse_drive, jump_op , "mesolve", options=options)
                    
    return result   

In [36]:
start = time.time()

simu_opt = qtp.solver.Options()
simu_opt.atol = 1e-12
simu_opt.rtol = 1e-10

_proj_comp = np.einsum('ai, aj -> ij', comp_states, np.conj(comp_states))
subsys_dims = list(q.dim_hilbert for q in coupled_sys)
proj_comp = qtp.Qobj(inpt=_proj_comp, dims=[subsys_dims, subsys_dims], isherm=True)
res = run(time=2*np.pi*t_list, system=coupled_sys, state_init=proj_comp/d_comp, with_noise=True, options=simu_opt)

end=time.time()
display(Latex(r'$\mathrm{{Leakage \, computation \, time}} = {:.3f} \, s$'.format(end - start)))
l1 = 1 - qtp.expect(proj_comp, res.states[-1])

<IPython.core.display.Latex object>

In [37]:
leakage_states = ['13', '03', '04', '12', '02']

In [38]:
def get_probabilities(
    states_label: List[str], 
    system:QubitSystem, 
    final_state: qtp.qobj.Qobj
) -> Dict:
    prob = {}
    for label in states_label:
        ket = system.state(label, as_qobj=True)[1]
        projector = ket*ket.dag()
        prob[label] = qtp.expect(projector, final_state)
    return prob
    

In [39]:
leak_probs = get_probabilities(leakage_states, coupled_sys, res.states[-1])

display(Latex(r'$\mathrm{{Average\, Leakage \,}} L_1 = {:.7f} $'.format(l1)))

sum_leak = 0

for label in leakage_states:
    display(Latex(r'$P_{{leak, {}  }} = {:.7f}$'.format(label, leak_probs[label])))
    sum_leak += leak_probs[label]
display(Latex(r'$\sum_k P_{{leak, k  }} = {:.7f}$'.format(sum_leak)))

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>